In [1]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.2).
ffmpeg is already the newest version (7:3.4.6-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 4 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 2s (92.2 kB/s)
Select

In [2]:
!pip install pyaudio

  Created wheel for pyaudio: filename=PyAudio-0.2.11-cp36-cp36m-linux_x86_64.whl size=51632 sha256=ceddbb6023ed41abb023fddd692da61ef37af1c84f3ae41c1fa9aaab38b96b39
  Stored in directory: /root/.cache/pip/wheels/f4/a8/a4/292214166c2917890f85b2f72a8e5f13e1ffa527c4200dcede
Successfully built pyaudio


In [0]:
import pyaudio
import wave
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms



In [0]:
class Net(nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1=nn.Conv2d(1,20,5,1)
    self.conv2=nn.Conv2d(20,50,5,1)
    self.fc1=nn.Linear(4*4*50,500)
    self.fc2=nn.Linear(500,10)
    
    self.ordered_layers=[self.conv1,self.conv2,self.fc1,self.fc2]
    
  def forward(self,x):
    x=F.relu(self.conv1(x))
    x=F.max_pool2D(x,2,2)
    x=F.relu(self.conv2(x))
    x=F.max_pool2D(x,2,2)
    x=x.view(-1,4*4*50)
    x=F.relu(self.fc1(x))
    x=self.fc2(x)
    return F.log_softmax(x,dim=1)

In [0]:
def open_stream(fs):
  p=pyaudio.PyAudio()
  stream=p.open(format=pyaudio.paFloat32,channels=1,rate=fs,output=True,output_device_index=0)
  return p,stream

In [0]:
def gen(fs,freq,duration):
  sinwave=np.sin(2*np.pi*np.arrange(fs*duration)*freq/fs)
  sample=sinwave.astype(np.float32)
  return 0.1*sample

In [0]:
def train(model,device,train_loader,optimizer,epoch):
  model.train()
  fs=44100
  duration=0.01
  f=200.0
  p,stream=open_stream(fs)
  frames=[]
  for batch_idx, (data, target) in enumerate(train_loader):
    data,target=data.to(device),target.to(device)
    optimizer.zero_grad()
    output=model(data)
    loss=F.nll_loss(output,target)
    loss.backward()
    norms=[]
    for layer in model.ordered_layers:
      norm_grad=layer.weight.grad.norm()
      norms.append(norm_grad)
      
      tone=f+(norms.numpy()*100.0)
      tone=tone.astype(np.float32)
      samples=gen(fs,tone,duration)
      
      frames.append(samples)
    
    silence=np.zeros(samples.shape[0]*2,dtyple=np.float32)
    frames.append(silence)
    optimizer.step()
    
    if batch_idx==150:
      break
      
  wf=wave.open("sgdlr 0.01.wav","wb")
  wf.setnchannels(1)
  wf.setsampwidth(p.get_sample_size(pyaudio.paFloat32))
  wf.setframerate(fs)
  wf.writeframes(b''.join(frames))
  wf.close()
  
  stream.stop_stream()
  stream.close()
  p.terminate()
  
  

In [0]:
train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=256, shuffle=True)

In [0]:
device=torch.device('cuda')
model=Net().to(device)

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [0]:
for epoch in range(1, 10):
        train(model, device, train_loader, optimizer, epoch)